In [26]:
import pandas as pd
import duckdb 

df = pd.read_excel(
    "data/resultat-2024-for-kurser-inom-yh (1).xlsx" 
)
df.head()

,Diarienummer,Beslut,Anordnare namn,Utbildningsnamn,Utbildningsområde,Antal beviljade platser start 2024,Antal beviljade platser start och slut 2024,Antal beviljade platser start 2025,Totalt antal beviljade platser,YH-poäng,Kommun,Län,FA-region,Antal kommuner,Antal län,Antal FA-regioner
0,MYH 2024/1000,Avslag,Changemaker Educations AB,Futuregames Accessible Game Production,Juridik,0,0,0,0,25,"Se ""Lista flera kommuner""","Se ""Lista flera kommuner""","Se ""Lista flera kommuner""",3,3,3
1,MYH 2024/1001,Avslag,Changemaker Educations AB,Futuregames Inclusive Game Development,"Ekonomi, administration och försäljning",0,0,0,0,25,"Se ""Lista flera kommuner""","Se ""Lista flera kommuner""","Se ""Lista flera kommuner""",3,3,3
2,MYH 2024/1002,Avslag,Båstads kommun Akademi Båstad,Ekosystemtjänster i praktiken,"Lantbruk, djurvård, trädgård, skog och fiske",0,0,0,0,50,Båstad,Skåne,Malmö-Lund,1,1,1
3,MYH 2024/1003,Avslag,Båstads kommun Akademi Båstad,Hållbar dagvattenhantering i praktiken,"Lantbruk, djurvård, trädgård, skog och fiske",0,0,0,0,50,Båstad,Skåne,Malmö-Lund,1,1,1
4,MYH 2024/1004,Avslag,Båstads kommun Akademi Båstad,Invasiva växter i utemiljöer,"Lantbruk, djurvård, trädgård, skog och fiske",0,0,0,0,25,Båstad,Skåne,Malmö-Lund,1,1,1


In [28]:
df_regions = duckdb.query(
    """
    SELECT 
        län, 
        COUNT_IF(beslut = 'Beviljad') AS Beviljade
    FROM df 
    WHERE 
        län != 'Flera kommuner'
        AND län != 'Se "Lista flera kommuner"'
    GROUP BY
        län
    ORDER BY 
        Beviljade DESC
    """
).df()

df_regions


,Län,Beviljade
0,Stockholm,143.0
1,Västra Götaland,87.0
2,Skåne,43.0
3,Örebro,14.0
4,Västmanland,13.0
5,Västerbotten,10.0
6,Södermanland,10.0
7,Östergötland,8.0
8,Uppsala,8.0
9,Kronoberg,7.0


In [53]:
import json
from difflib import get_close_matches

with open("../assets/swedish_regions.geojson", "r") as file:
    json_data = json.load(file)

name = json_data.get("features")[6].get("properties")

name

{'admin_level': '4',
 'boundary': 'administrative',
 'name': 'VÃ¤stra GÃ¶talands lÃ¤n',
 'ref:se:lÃ¤n:bokstav': 'O',
 'ref:se:lÃ¤nskod': '14',
 'type': 'multipolygon'}

In [54]:
df_regions["Län"].values


array(['Stockholm', 'Västra Götaland', 'Skåne', 'Örebro', 'Västmanland',
       'Västerbotten', 'Södermanland', 'Östergötland', 'Uppsala',
       'Kronoberg', 'Värmland', 'Blekinge', 'Gävleborg', 'Västernorrland',
       'Jönköping', 'Norrbotten', 'Kalmar', 'Halland', 'Jämtland',
       'Dalarna'], dtype=object)

In [56]:
get_close_matches(name, df_regions["Län"], n=1)[0]

IndexError: list index out of range

In [38]:
import json
from difflib import get_close_matches

region_codes_map = []

for region in df_regions["Län"]:
    matches = get_close_matches(region, regions_codes.keys(), n=1)
    if matches:
        region_name = matches[0]
        code = regions_codes[region_name]
        region_codes_map.append(code)
    else:
        print(f"Ingen match hittad för: {region}")
        region_codes_map.append(None)  # eller något standardvärde

print(region_codes_map)

Ingen match hittad för: Skåne
Ingen match hittad för: Örebro
Ingen match hittad för: Jönköping
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [33]:
approved = df["Beslut"].value_counts()["Beviljad"]
approved

np.int64(451)

In [34]:
total_applications = df["Beslut"].value_counts().sum()
total_applications

np.int64(880)